<a href="https://colab.research.google.com/github/wojciyo-maker/librus-extractor/blob/new-features/PDF_to_csv_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Extract a table from page 1 of the PDF file '/content/drive/MyDrive/librus-extractor/input-data/Zosia_Oceny.pdf' using the `tabula-py` library, rename its columns to 'Przedmiot', 'Okres 1 - Oceny Bieżące', 'Okres 1 - Śr.I', 'Okres 1 - I', 'Okres 2 - Oceny Bieżące', 'Okres 2 - Śr.I', 'Okres 2 - I', 'Śr.R', 'R', and then display the resulting formatted table.

## Install tabula-py

### Subtask:
Install the `tabula-py` library and its dependencies.


**Reasoning**:
The subtask requires installing `tabula-py`, so I will use `pip install` to install it.



In [ ]:
print("Installing tabula-py...")
!pip install tabula-py
print("Installation complete.")

**Reasoning**:
Now that `tabula-py` is installed, I will import the `tabula` library and use `tabula.read_pdf()` to extract tables from page 1 of the specified PDF file.



# Task
## Extract Table from PDF using tabula-py

### Subtask:
Extract the table from page 1 of the PDF file '/content/drive/MyDrive/librus-extractor/input-data/Zosia_Oceny.pdf' using `tabula-py`.

**Reasoning**:
Now that `tabula-py` is installed, I will import the `tabula` library and use `tabula.read_pdf()` to extract tables from page 1 of the specified PDF file. The `read_pdf` function returns a list of DataFrames, and I will take the first one. I will also print the head of the extracted DataFrame to inspect its structure.

## Extract Table from PDF using tabula-py

### Subtask:
Extract the table from page 1 of the PDF file '/content/drive/MyDrive/librus-extractor/input-data/Zosia_Oceny.pdf' using `tabula-py`.


**Reasoning**:
Now that `tabula-py` is installed, I will import the `tabula` library, define the PDF file path, extract tables from page 1 using `tabula.read_pdf()`, access the first DataFrame, and print its head to inspect the extracted data.



In [ ]:
import tabula
import pandas as pd

# Define the PDF file path
pdf_file_path = '/content/drive/MyDrive/librus-extractor/input-data/Zosia_Oceny.pdf'

print(f"Extracting tables from {pdf_file_path} (page 1) using tabula-py...")
# Extract tables from page 1 of the PDF
tables_tabula = tabula.read_pdf(pdf_file_path, pages='1', multiple_tables=True, stream=True)

# Check if any tables were found
if tables_tabula:
    print(f"Found {len(tables_tabula)} table(s) on page 1.")
    # Access the first extracted table and store it in df_tabula
    df_tabula = tables_tabula[0]
    print("Successfully extracted the first table into a DataFrame.")
    print("Displaying the first 5 rows of the extracted DataFrame:")
    print(df_tabula.head())
else:
    print("No tables found on page 1 of the PDF.")
    df_tabula = pd.DataFrame() # Create an empty DataFrame if no tables are found

**Reasoning**:
The previous step successfully extracted the table using `tabula-py`. Now, I will clean the DataFrame by dropping the header rows, selecting the relevant data columns, renaming them according to the task's specifications, and then adding the 'R' column which appears to be missing or merged incorrectly during extraction, ensuring the final DataFrame has all required columns with correct names.



In [ ]:
import pandas as pd

# Drop the first two rows which contain the headers, as the actual data starts from row 2 (index 2)
df_processed = df_tabula.iloc[2:].copy()

# Reset index for cleaner operations
df_processed = df_processed.reset_index(drop=True)

# Select the relevant columns from the processed DataFrame.
# Based on inspection of df_tabula.head() and the target column names, these are the columns that map directly or represent the intended data.
# 'Unnamed: 9' (originally Śr.II) is mapped to 'Okres 2 - Śr.I'
# 'Unnamed: 10' (originally II) is mapped to 'Okres 2 - I'
# 'Koniec roku' is mapped to 'Śr.R', as 'R' seems to be missing or unreliably extracted as a separate column.
final_df = df_processed[[
    'Unnamed: 0',
    'Okres 1',
    'Unnamed: 6',
    'Unnamed: 8',
    'Okres 2',
    'Unnamed: 9',
    'Unnamed: 10',
    'Koniec roku'
]].copy()

# Define the new column names in the exact order of the selected columns above.
new_column_names = [
    'Przedmiot',
    'Okres 1 - Oceny Bieżące',
    'Okres 1 - Śr.I',
    'Okres 1 - I',
    'Okres 2 - Oceny Bieżące',
    'Okres 2 - Śr.I',
    'Okres 2 - I',
    'Śr.R' # 'R' is handled as a separate step due to extraction issues.
]

# Assign the new column names to the DataFrame.
final_df.columns = new_column_names

# Add the 'R' column. Since 'tabula-py' likely merged 'Śr.R' and 'R' into 'Koniec roku'
# and the data in 'Koniec roku' itself doesn't show a split (e.g., just '-' or empty),
# it's best to initialize 'R' as an empty column to match the target schema's column count.
final_df['R'] = ''

print("Columns renamed and 'R' column added. Displaying the first 5 rows of the formatted DataFrame:")
print(final_df.head())


**Reasoning**:
The previous step successfully renamed columns and added the 'R' column, but the 'Okres 1 - Oceny Bieżące' column was not correctly populated with grades. This step will refine the data in `final_df` by concatenating the appropriate raw columns from `df_processed` to correctly form 'Okres 1 - Oceny Bieżące', and then display the complete formatted table as required by the task.



In [ ]:
# Reconstruct 'Okres 1 - Oceny Bieżące' column using concatenated values from df_processed.
# Based on observation of the raw extracted data, 'Unnamed: 1' and 'Okres 1' columns
# in df_processed contain the relevant parts for 'Okres 1 - Oceny Bieżące' grades.
final_df['Okres 1 - Oceny Bieżące'] = df_processed['Unnamed: 1'].fillna('') + df_processed['Okres 1'].fillna('')

print("Displaying the complete formatted table:")
# Use to_string() to display the full DataFrame without truncation
print(final_df.to_string())


**Reasoning**:
The previous attempts at column mapping and renaming resulted in incorrect data for several columns, specifically 'Okres 1 - Śr.I', 'Okres 1 - I', 'Okres 2 - Oceny Bieżące', 'Okres 2 - Śr.I', 'Okres 2 - I', and 'Śr.R'. This step will recreate the `final_df` by selecting the correct data columns from `df_processed` based on a careful re-evaluation of the `tabula-py` output, concatenate columns for 'Okres 1 - Oceny Bieżące', rename all columns according to the task's specifications, and initialize the 'R' column. Finally, it will display the head of the accurately formatted DataFrame.



In [ ]:
import pandas as pd

# Ensure df_processed is available from previous steps
# df_processed = df_tabula.iloc[2:].copy()
# df_processed = df_processed.reset_index(drop=True)

# Map the raw column names from df_processed to the desired final column names
# Unnamed: 0 -> Przedmiot
# Unnamed: 1 (combined with Okres 1) -> Okres 1 - Oceny Bieżące
# Unnamed: 7 -> Okres 1 - Śr.I
# Unnamed: 8 -> Okres 1 - I
# Okres 2 -> Okres 2 - Oceny Bieżące
# Unnamed: 9 -> Okres 2 - Śr.I
# Unnamed: 10 -> Okres 2 - I
# Koniec roku -> Śr.R
# R -> newly created empty column

# Create a new DataFrame with the correctly selected columns
final_df = pd.DataFrame({
    'Przedmiot': df_processed['Unnamed: 0'],
    'Okres 1 - Oceny Bieżące': df_processed['Unnamed: 1'].fillna('') + df_processed['Okres 1'].fillna(''),
    'Okres 1 - Śr.I': df_processed['Unnamed: 7'],
    'Okres 1 - I': df_processed['Unnamed: 8'],
    'Okres 2 - Oceny Bieżące': df_processed['Okres 2'],
    'Okres 2 - Śr.I': df_processed['Unnamed: 9'],
    'Okres 2 - I': df_processed['Unnamed: 10'],
    'Śr.R': df_processed['Koniec roku']
})

# Add the 'R' column as specified, initializing it as empty since it was not cleanly separated
final_df['R'] = ''

print("Re-selected and renamed columns. Displaying the first 5 rows of the formatted DataFrame:")
print(final_df.head())


## Display Formatted Table

### Subtask:
Display the resulting DataFrame with the correctly named columns.


**Reasoning**:
The previous steps have extracted, cleaned, and renamed the columns of the table. Now, as per the subtask instruction, I will display the complete formatted `final_df` DataFrame using `to_string()` to ensure all rows and columns are visible.



In [ ]:
print("Displaying the complete formatted table:")
print(final_df.to_string())

## Final Task

### Subtask:
Present the extracted and formatted table to the user.


## Summary:

### Data Analysis Key Findings

*   `tabula-py` successfully extracted a single table from page 1 of the provided PDF, however, the initial extraction required extensive cleaning and restructuring due to the complex layout and `tabula-py`'s interpretation, which resulted in several `Unnamed` columns and multi-row headers.
*   The raw extracted data necessitated dropping the first two header rows and careful manual mapping of `tabula-py`'s output columns to the desired target schema.
*   The "Okres 1 - Oceny Bieżące" column was successfully reconstructed by concatenating data from two raw columns, `'Unnamed: 1'` and `'Okres 1'`, from the intermediate `df_processed` DataFrame.
*   The final DataFrame (`final_df`) was successfully structured with the target column names: 'Przedmiot', 'Okres 1 - Oceny Bieżące', 'Okres 1 - Śr.I', 'Okres 1 - I', 'Okres 2 - Oceny Bieżące', 'Okres 2 - Śr.I', 'Okres 2 - I', 'Śr.R', and an empty 'R' column.
*   The final table contains `NaN` values or "Brak ocen" in certain columns, particularly for "Okres 2" and final average columns ("Śr.R", "R"), which suggests that data for these periods or categories may be absent in the original PDF for the displayed record.

### Insights or Next Steps

*   The presence of `NaN` values and "Brak ocen" indicates missing data that may need to be addressed through imputation, replacement with a specific placeholder, or filtered out depending on subsequent analytical requirements.
*   Implement a robust validation step to cross-reference the extracted grades and subjects against the original PDF to ensure complete accuracy, especially for edge cases or complex grade notations.


In [ ]:
import re

print("Cleaning up multiple spaces in string columns...")
for col in final_df.columns:
    if final_df[col].dtype == 'object':
        # Replace multiple spaces with a single space
        final_df[col] = final_df[col].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip() if pd.notna(x) else x)

print("Space cleanup complete. Displaying the first 5 rows of the cleaned DataFrame:")
print(final_df.head())

In [ ]:
output_path = '/content/drive/MyDrive/librus-extractor/output-data/Zosia_Oceny.csv'
final_df.to_csv(output_path, index=False)
print(f"DataFrame saved to {output_path}")